In [ ]:
# Example - https://github.com/cornelliusyudhawijaya/Churn_Causality_Analysis/blob/main/Causal%20Analysis%20Do%20Why.ipynb
    
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../../../data/Titanic.csv')

## Drop categorical features
data = data.drop(['Name','Ticket','Cabin','PassengerId','Embarked'],axis=1)

## Drop NULL values
data = data.dropna()

## Drop NULL values
data = data.fillna(0)

## Change symbolics to numerics
data['sex'] = np.where(data['sex'] == 'male', 1, 0)

protected_attribute = 'sex'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)
print(data.head(5))

In [ ]:
# This dataset size is very small. So repeating it to make it bigger

data = pd.concat([data]*5, ignore_index=True)

# Transform Data using Reweighing

In [ ]:
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.datasets import BinaryLabelDataset

dataset = BinaryLabelDataset(df=data, label_names=['Probability'], protected_attribute_names=[protected_attribute])

privileged_groups = [{protected_attribute: 1}]
unprivileged_groups = [{protected_attribute: 0}]

RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

RW.fit(dataset)
dataset_transf = RW.transform(dataset)

df, weights = dataset_transf.convert_to_dataframe()

df.head(3)

In [ ]:
index = 0

for i,row in df.iterrows():
    duplicate = round(weights["instance_weights"][int(index)])*10
    for j in range(duplicate):
        df = df.append(row)
    index += 1
print(df.shape)

In [ ]:
causal_graph = """
digraph {
Age;
sex;
Fare;
Probability;
U[label="Unobserved Confounders"];
Age -> Probability;
sex -> Probability;
sex -> Fare; Fare -> Probability;
U->sex;U->Probability;
}
"""

## dowhy works for binary inputs only

df['Probability'] = np.where(df['Probability'] == 0, False, True)
df[protected_attribute] = np.where(df[protected_attribute] == 1, True, False)

In [ ]:
# With graph
model=CausalModel(
        data = df,
        treatment=protected_attribute,
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )
model.view_model()

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)